![image](https://www.ewatercycle.org/assets/logo.png)

In [32]:
from pathlib import Path
import time
import shapely as shply
import shapely.vectorized as shply_vect
import xarray as xr
from geopandas import GeoSeries
import numpy as np
import shapely as shply
import fiona

from esmvalcore.experimental import CFG
from ewatercycle import forcing
from ewatercycle.models import Lisflood
from ewatercycle.models.lisflood import reindex_forcings
from ewatercycle.models.lisflood import LisfloodParameterSet

In [33]:
mask = Path("/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/LISFLOOD/catchment_masks.nc")
TEMP_DIR = Path(f"/scratch/shared/ewatercycle/lisflood_{time.strftime('%Y%m%d_%H%M%S')}")
TEMP_DIR.mkdir(parents=True, exist_ok=True)
print(TEMP_DIR)

/scratch/shared/ewatercycle/lisflood_20210413_182908


In [36]:
# TODO Compute convex hull
masks = xr.open_dataarray(mask).load()
buffer = 0.05  # degrees lat/lon
catchment = 'Meuse'
# Compute convex hull
lat, lon = [
    masks[v].values[np.where(masks.loc[catchment].values)[i]]
    for v, i in zip(["lat", "lon"], [0, 1])
]
hull = GeoSeries(
    [shply.geometry.Point(x, y) for x, y in zip(lon, lat)]
).unary_union.convex_hull.buffer(buffer)
# Check that all pixels are inside the hull
assert all(shply_vect.contains(hull, lon, lat))
# Save it to shapefile in temp dir
shapefile = TEMP_DIR / f"{catchment}.shp"
GeoSeries(hull, crs="EPSG:4326").to_file(shapefile)
print(
    "Convex hull of {} basin saved to shapefile {}".format(
        catchment, shapefile
    )
)

Convex hull of Meuse basin saved to shapefile /scratch/shared/ewatercycle/lisflood_20210413_182908/Meuse.shp


<ipython-input-36-0570c0410f97>:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [37]:
#TODO get extent for extract region in the recipe
def get_extents(shapefile, pad=0):
    """Get lat/lon extents from shapefile and add padding."""
    shape = fiona.open(shapefile)
    x0, y0, x1, y1 = [
        shply.geometry.shape(p["geometry"]).bounds for p in shape
    ][0]
    x0 = round((x0 - pad), 1)
    y0 = round((y0 - pad), 1)
    x1 = round((x1 + pad), 1)
    y1 = round((y1 + pad), 1)
    return x0, y0, x1, y1
x0, y0, x1, y1 = get_extents(shapefile)


In [38]:
# load config file
CFG.load_from_file('~/.esmvaltool/config-user.yml')

In [39]:
# forcing.generate takes a single forcing dataset
forcing_output = forcing.generate(
    model='lisflood', 
    dataset='ERA-Interim',
    startyear=1990, 
    endyear=1990,
    shapefile=shapefile,
    extract_region={
        'start_longitude': y0,
        'end_longitude': y1,
        'start_latitude': x0,
        'end_latitude': x1,
    }
)

{'auxiliary_data_dir': PosixPath('/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/Lorentz_Basin_Shapefiles'),
 'compress_netcdf': False,
 'config_developer_file': None,
 'config_file': PosixPath('/home/fakhereh/.esmvaltool/config-user.yml'),
 'drs': {'CMIP5': 'default', 'CMIP6': 'default'},
 'exit_on_warning': False,
 'log_level': 'info',
 'max_parallel_tasks': 1,
 'output_dir': PosixPath('/scratch-shared/ewatercycle'),
 'output_file_type': 'png',
 'plot_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_163147/plots'),
 'preproc_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_163147/preproc'),
 'profile_diagnostic': False,
 'remove_preproc_dir': True,
 'rootpath': {'OBS6': [PosixPath('/lustre1/0/wtrcycle/comparison/obs6')]},
 'run_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_163147/run'),
 'save_intermediary_cubes': False,
 'work_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_163147/wor

In [40]:
list(forcing_output.recipe_output.items())

[('diagnostic_daily/script',
  diagnostic_daily/script:
    DataFile('lisflood_ERA-Interim_Meuse_sfcWind_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_rsds_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_e_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_tasmin_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_tas_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_pr_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_tasmax_1990_1990.nc'))]

In [41]:
# reindex forcing to global extent
reindex_forcings(mask, forcing_output, TEMP_DIR)

PosixPath('/scratch/shared/ewatercycle/lisflood_20210413_182908')

## Run lisflood class

In [42]:
parameterset = LisfloodParameterSet(
    # TODO one level down (+ /Lisflood01degree_masked)?
    root=Path('/projects/0/wtrcycle/comparison/lisflood_input'),
    # TODO dir or file (+ /model_mask.nc)?
    mask=mask.parent,
    config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisflood.xml'),
    lisvap_config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisvap.xml'),
)

In [43]:
# create lisflood instance
model = Lisflood()

In [45]:
# setup model
FORCING_DIR = Path('path_to_one_year_data') # from comparison study or
FORCING_DIR = TEMP_DIR
config_file, config_dir = model.setup(FORCING_DIR, parameterset)

Running ewatercycle-lisflood-grpc4bmi.sif singularity container on port 32905


In [46]:
print(config_file)
print(config_dir)

/scratch/shared/ewatercycle/lisflood_20210413_191039/lisflood_setting_1990_1990.xml
/scratch/shared/ewatercycle/lisflood_20210413_191039


In [47]:
# get metadata
print(model.output_var_names)

('Discharge',)


In [48]:
# run lisvap
model.run_lisvap(FORCING_DIR)

(0,
 b'Lisvap  0.4.2   23/08/2019\n\n(C) Disaster Risk Management Knowledge Centre\n    Joint Research Centre of the European Commission\n    Units E1,D2 I-21020 Ispra (Va), Italy\n\n\n\nStart date: 01/01/1990 00:00 (1) - End date: 31/12/1990 00:00 (365)\n\r1\r2\r3\r4\r5\r6\r7\r8\r9\r10\r11\r12\r13\r14\r15\r16\r17\r18\r19\r20\r21\r22\r23\r24\r25\r26\r27\r28\r29\r30\r31\r32\r33\r34\r35\r36\r37\r38\r39\r40\r41\r42\r43\r44\r45\r46\r47\r48\r49\r50\r51\r52\r53\r54\r55\r56\r57\r58\r59\r60\r61\r62\r63\r64\r65\r66\r67\r68\r69\r70\r71\r72\r73\r74\r75\r76\r77\r78\r79\r80\r81\r82\r83\r84\r85\r86\r87\r88\r89\r90\r91\r92\r93\r94\r95\r96\r97\r98\r99\r100\r101\r102\r103\r104\r105\r106\r107\r108\r109\r110\r111\r112\r113\r114\r115\r116\r117\r118\r119\r120\r121\r122\r123\r124\r125\r126\r127\r128\r129\r130\r131\r132\r133\r134\r135\r136\r137\r138\r139\r140\r141\r142\r143\r144\r145\r146\r147\r148\r149\r150\r151\r152\r153\r154\r155\r156\r157\r158\r159\r160\r161\r162\r163\r164\r165\r166\r167\r168\r169\r170\r

In [49]:
# initialize
model.initialize(config_file)

In [50]:
# run model with bmi
model.update()

In [51]:
# get value
model.get_value('Discharge')

Too many items (5400000) for single call, using multiple get_value_at_indices() with into chunks of 524280 items
Fetching value range 0 - 524280
Fetching value range 524280 - 1048560
Fetching value range 1048560 - 1572840
Fetching value range 1572840 - 2097120
Fetching value range 2097120 - 2621400
Fetching value range 2621400 - 3145680
Fetching value range 3145680 - 3669960
Fetching value range 3669960 - 4194240
Fetching value range 4194240 - 4718520
Fetching value range 4718520 - 5242800
Fetching value range 5242800 - 5400000


array([nan, nan, nan, ..., nan, nan, nan])

In [52]:
# stop the model
model.finalize()
del model.bmi

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: 'LisfloodBmi' object has no attribute '_userModel'"
	debug_error_string = "{"created":"@1618335248.141257312","description":"Error received from peer ipv6:[::1]:32905","file":"src/core/lib/surface/call.cc","file_line":1067,"grpc_message":"Exception calling application: 'LisfloodBmi' object has no attribute '_userModel'","grpc_status":2}"
>